In [5]:
import pandas as pd 

Сделаем для датасета с 2016 года и поглядим вообще на то, что имеем по длинам серий

In [8]:
df_cur = pd.read_excel("witn_new_rat_ag_num_cat_catnum.xlsx")


In [9]:
df_cur.head(10)

,Unnamed: 0,_id,success,_name,name_comp,_ro_type,identifier,identifier_type,additional_identifier_type,isin,...,id_uuid,json_additional,meta,new_rating,agency,rating_numeric,rating_numeric_int,rating_category,rating_category_,rating_numeric_cat
0,0,1,True,Общество с ограниченной ответственностью Холди...,NaN,BNFC - нефинансовая компания,7801293231,ИНН,NaN,NaN,...,a0f6e8c8-a2e7-4740-97ac-fff125c0821a,NaN,"{'Comment': 'Success', 'added key to': 'level_...",ruBBB-,Expert RA,10,10,BBB,BBB,4
1,1,2,True,Общество с ограниченной ответственностью ПЕТЕР...,NaN,BNFC - нефинансовая компания,7801423924,ИНН,NaN,NaN,...,c317bce9-7d26-456a-be8c-6f2cda45042d,NaN,"{'Comment': 'Success', 'added key to': 'level_...",ruB+,Expert RA,14,14,B,B,6
2,2,3,True,Общество с ограниченной ответственностью ХРОМО...,NaN,BNFC - нефинансовая компания,5249111131,ИНН,NaN,NaN,...,9643b9cf-295f-4fc9-82d4-30f3b791101f,NaN,"{'Comment': 'Success', 'added key to': 'level_...",ruBB,Expert RA,12,12,BB,BB,5
3,3,5,True,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""КИНГ...",NaN,BNFC - нефинансовая компания,7816562511,ИНН,NaN,NaN,...,537699a6-7c72-46a4-b2bd-b12b7157fe3e,NaN,"{'Comment': 'Success', 'added key to': 'level_...",BBB-(RU),AKRA,10,10,BBB,BBB,4
4,4,1634,True,"Облигации, АО «АЛЬФА-БАНК», Старший необеспече...",АКЦИОНЕРНОЕ ОБЩЕСТВО «АЛЬФА-БАНК»,TBND - облигационный займ,7728168971,ИНН,NaN,RU000A107A69,...,d87bb094-ca4b-42f5-967a-be180b943d96,NaN,"{'Comment': 'Success', 'added key to': 'level_...",AA+(RU),AKRA,2,2,AA,AA,2
5,5,1737,True,"Облигации, АО «АЛЬФА-БАНК», Старший необеспече...",NaN,TBND - облигационный займ,7728168971,ИНН,NaN,RU000A0JX5W4,...,3a53d6f6-a7e3-442c-a9e3-4f97b72a74a5,NaN,"{'Comment': 'Success', 'added key to': 'level_...",AA+(RU),AKRA,2,2,AA,AA,2
6,6,1738,True,"Облигации, АО «АЛЬФА-БАНК», Старший необеспече...",NaN,TBND - облигационный займ,7728168971,ИНН,NaN,RU000A0JX5W4,...,2f5c1a11-2c93-46f7-b050-a5e93fb793d8,NaN,"{'Comment': 'Success', 'added key to': 'level_...",AA(RU),AKRA,3,3,AA,AA,2
7,7,1739,True,"Облигации, АО «АЛЬФА-БАНК», Старший необеспече...",NaN,TBND - облигационный займ,7728168971,ИНН,NaN,RU000A0JX5W4,...,d625b597-2646-449f-9ddf-f0f28840d22e,NaN,"{'Comment': 'Success', 'added key to': 'level_...",AA(RU),AKRA,3,3,AA,AA,2
8,8,1803,True,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""РАЗРЕЗ КОЛЫВАНСКИЙ""",NaN,BNFC - нефинансовая компания,5406192366,ИНН,NaN,NaN,...,0c46e084-c183-4536-97aa-9fbab07ecd78,NaN,"{'Comment': 'Success', 'added key to': 'level_...",A(RU),AKRA,6,6,A,A,3
9,9,7,True,"Публичное акционерное общество ""АВТОКОМБИНАТ №...",NaN,CO - прочий вид организации,7728013537,ИНН,NaN,NaN,...,2491b9ba-128c-419e-854b-46a03a76d10a,NaN,"{'Comment': 'Success', 'added key to': 'level_...",ruB,Expert RA,15,15,B,B,6


Убираем все наблюдения с облигациями, так как они не относятся к тому, что нам надо

In [11]:
df_cur = df_cur.loc[~df_cur["_ro_type"].str.startswith("T", na=False)]


In [12]:
df_cur["agency"].unique()

array(['Expert RA', 'AKRA', 'NCR', 'NRA'], dtype=object)

In [13]:
ident = df_cur["identifier"].unique()

In [14]:
len(ident)

1306

In [15]:
series_length = {}
for inn in ident:
    if inn not in series_length:
        series_length[inn] = {}
    for agency in df_cur["agency"].unique():
        if agency not in series_length[inn]:
            series_length[inn][agency] = []
        temp_df = df_cur[(df_cur["identifier"] == inn) & (df_cur["agency"] == agency)]
        curr_len = len(temp_df["new_rating"])
        series_length[inn][agency] = curr_len   # there should apply the length of series of each company in cra
    

In [16]:
df_ser = pd.DataFrame().from_dict(series_length).T.reset_index()

In [17]:
df_ser.rename(columns={"index": "identifier"}, inplace=True)

In [18]:
df_ser[df_ser["Expert RA"] > 10]

,identifier,Expert RA,AKRA,NCR,NRA
188,7805104870,11,0,0,1
265,7708022300,11,1,0,0
294,5047254063,11,0,0,0
299,5902300072,11,0,4,0
304,7704627560,12,0,0,0
317,3328409738,11,0,0,1
326,1835047032,12,0,0,3
332,7744000302,12,8,0,0
345,268008010,13,0,0,0
379,7727065444,11,0,0,0


In [19]:
df_ser[df_ser["AKRA"] > 10]

,identifier,Expert RA,AKRA,NCR,NRA
12,3801002781,1,13,0,3
190,7724490000,0,16,2,0
251,5722033117,2,16,0,0
254,5503067018,0,12,0,0
303,7703213534,4,11,0,3
493,7704274402,14,14,0,0
576,7700076777,0,11,0,0
667,7612031791,2,12,0,0
743,7706092528,9,13,7,0


In [20]:
df_ser[df_ser["NCR"] > 10]

,identifier,Expert RA,AKRA,NCR,NRA


In [21]:
df_ser[df_ser["NRA"] > 10]

,identifier,Expert RA,AKRA,NCR,NRA


Всего мы имеем порядка 35 компаний, у которыйх наблюдений больше 35 10 у Эксперта и 9 у АКРЫ

Аналогично посмотрим для другого датасета до 2016 года и попробуем как - то его смержить с исходным

In [24]:
df_last = pd.read_excel("/Users/danilalipatov/PycharmProjects/diploma/data/TO_WORK_WITH_final_last_new.xlsx")

In [25]:
df_last.columns

Index(['Unnamed: 0', 'ind', '_id', 'agency', '_ro_type', '_name', 'identifier',
       'identifier_type', 'additional_identifier',
       'additional_identifier_type', '_date', 'rating', 'status',
       'additional_industry', 'company_link', 'forecast', 'press_release_text',
       'press_release_link', '_comment', 'scale', 'ogrn', 'fin_instrument',
       'uuid_from_level_1', 'id_uuid', 'json_additional', 'Added_default'],
      dtype='object')

In [26]:
len(df_last["identifier"].unique())

2250

In [32]:
counter_inn = []
for inn in df_last["identifier"].unique():
    if inn in ident:
        counter_inn.append(inn)
    if str(inn) in ident:
        counter_inn.append(inn)

In [34]:
len(counter_inn)

85

In [36]:
series_length = {}
for inn in df_last["identifier"].unique():
    if inn not in series_length:
        series_length[inn] = {}
    for agency in df_last["agency"].unique():
        if agency not in series_length[inn]:
            series_length[inn][agency] = []
        temp_df = df_last[(df_last["identifier"] == inn) & (df_last["agency"] == agency)]
        curr_len = len(temp_df["rating"])
        series_length[inn][agency] = curr_len   # there should apply the length of series of each company in cra

In [37]:
df_ser = pd.DataFrame().from_dict(series_length).T.reset_index()

In [38]:
df_ser.rename(columns={"index": "identifier"}, inplace=True)

Выберем наиболее подходящие из первого агенства

In [110]:
check = df_ser[df_ser["Expert RA"] >= 10]

In [112]:
for inn in check["identifier"]:
    print(inn, type(inn))

7710026574 <class 'str'>
7702073683 <class 'str'>
7710045520 <class 'str'>
7730042285 <class 'str'>
7705042179 <class 'str'>
7453297458 <class 'str'>
7744000038 <class 'str'>
1657023630 <class 'str'>
7702045051 <class 'str'>
8901010104 <class 'str'>
7705041231 <class 'str'>
7706196090 <class 'str'>
0411063374 <class 'str'>
7736035485 <class 'str'>
7831000210 <class 'str'>
7734202860 <class 'str'>
7712044762 <class 'str'>
7707284568 <class 'str'>
7702045615 <class 'str'>
1644001196 <class 'str'>
7709031643 <class 'str'>
7831001567 <class 'str'>
7703032986 <class 'str'>
7710280644 <class 'str'>
7725497022 <class 'str'>
7744001497 <class 'str'>
5027089703 <class 'str'>
7707067683 <class 'str'>
7834002576 <class 'str'>
7831000027 <class 'str'>
7704041020 <class 'str'>
8602103061 <class 'str'>
7703104630 <class 'str'>
3804002162 <class 'str'>
7704082517 <class 'str'>
7736046991 <class 'str'>
7733013853 <class 'str'>
7706107510 <class 'str'>
8602190258 <class 'str'>
7709129705 <class 'str'>


Так как разные типы данных в двух датасетах, надо выбрать пересечения 


In [115]:
for inn in ident:
    print(inn, type(inn))

7801293231 <class 'numpy.int64'>
7801423924 <class 'numpy.int64'>
5249111131 <class 'numpy.int64'>
7816562511 <class 'numpy.int64'>
5406192366 <class 'numpy.int64'>
7728013537 <class 'numpy.int64'>
5262146310 <class 'numpy.int64'>
9724021393 <class 'numpy.int64'>
277077282 <class 'numpy.int64'>
5407245973 <class 'numpy.int64'>
7710048970 <class 'numpy.int64'>
7728168971 <class 'numpy.int64'>
3801002781 <class 'numpy.int64'>
7714877093 <class 'numpy.int64'>
6829000028 <class 'numpy.int64'>
4706006731 <class 'numpy.int64'>
7725038220 <class 'numpy.int64'>
7303024532 <class 'numpy.int64'>
7423004062 <class 'numpy.int64'>
7701699414 <class 'numpy.int64'>
7703032986 <class 'numpy.int64'>
6315232133 <class 'numpy.int64'>
5036011870 <class 'numpy.int64'>
7736203789 <class 'numpy.int64'>
7714572888 <class 'numpy.int64'>
7705869699 <class 'numpy.int64'>
7831001013 <class 'numpy.int64'>
7750005450 <class 'numpy.int64'>
9726064201 <class 'numpy.int64'>
7838395215 <class 'numpy.int64'>
5404373487 

In [117]:
counter = 0
unique = []
for inn in ident:
    if str((inn)) in check["identifier"].values:
        print(inn)
        counter += 1
        unique.append(str(inn))
counter

7423004062
7703032986
6315232133
5405114781
8602190258
7744000951
8601999494
7831000122
2901009852
5502051657
7750004312
5902162552
6325064223
7536002161
7702073683
1700000350
7802084569
7728668727
7750005820
1326021671
7706628777
3525149310
5902198365
7709031643
7750004270
7725039953
1657023630
1433009894
7704300571
8602998609
7726445147
5902220892
7701100510
7815025049
7707050464
7707684460
2130058291
2460030985
2202000656
7449006730
7701041336
1326024785
7705880068
2801023444
1653003834
7728534829
8601023568
7725038124
7713056834
1650072068
2903000446
7744000038
1644001196
7709315684
7744001024
8904002359
7723811155
5902300072
7750056688
7704627560
7729399756
7812016906
7750005482
7705568878
3328409738
1435159327
8601027509
1835047032
6453033870
6625000100
3525030681
7709056550
6608003052
3327100351
3525023780
7736035485
6905011218
5407197984
7710140679
2204000595
5253004326
6316028910
7750004009
5200000046
7727499177
7727065444
7727499184
9725000621
7703379402
7709445387
7725352740

309

In [119]:
counter

309

Итого: пересечения по 309 компании в общих чертах , выберем из них самые большие:

In [125]:
check.columns

Index(['identifier', 'S&P Global Ratings', 'Expert RA', 'Fitch Ratings',
       'Moody's Investors Service', 'Moody's Interfax Rating Agency', 'AKRA',
       'A. M. Best', 'Dagong Global', 'Rating-Agentur Expert RA',
       'RAEX-Eroupe', 'China Lianhe Credit Ratings', 'NRA',
       'China Chengxin International Credit Rating (CCXI)'],
      dtype='object')

In [127]:
res = {}
for inn in unique:
    temp = df_last[(df_last["identifier"] == inn) & (df_last["agency"] == "Expert RA")]
    if inn not in res:
        res[inn] = []
    res[inn] = len(temp["rating"])


Общее кол-во наблюдений с учетом длинных серий

In [129]:
sum(list(res.values()))

4484

Аналогично посмотрим на данные,где состояний меньше 10

In [96]:
check = df_ser[df_ser["Expert RA"] < 10]
counter = 0
unique = []
for inn in ident:
    if str((inn)) in check["identifier"].values:
        print(inn)
        counter += 1
        unique.append(str(inn))
res = {}
for inn in unique:
    temp = df_last[(df_last["identifier"] == inn) & (df_last["agency"] == "Expert RA")]
    if inn not in res:
        res[inn] = []
    res[inn] = len(temp["rating"])
sum_ = sum(list(res.values()))

5406192366
7728013537
7710048970
7728168971
3801002781
7714877093
6829000028
4706006731
7725038220
7303024532
7701699414
5036011870
7736203789
7714572888
7705869699
7831001013
7750005450
7838395215
1435332211
2502039781
4715019631
7704612940
7809016423
6320007246
7704167721
7708063900
3812035570
7708813052
1435241444
7730231980
7202066550
2312054894
7725727149
6664013880
7750004231
7708709686
6454027396
7720413269
7705392230
7716051506
7727696432
7744000327
2204052762
5075011344
7731559044
5257116711
7703635416
7720261827
7702271396
3123011520
7708397772
7714056040
4345404971
7750004136
7750004030
7839502357
7731304818
5902300033
7730650445
7707056547
5012000639
5244009279
7710641442
1644039560
7705256396
7726581132
7734205131
7703033450
6686073603
7708678325
7813151859
7709345294
7825392577
7710924105
4004021785
5032001366
7801098872
7726637843
7711056412
7744001930
7701350084
7826092350
7707177911
5835103100
7731038186
9729278924
4716028445
7726391004
7705456910
7708001614
7717601826

Совпадения при меньше 10 и суммарное кол-во наблюдений: 

In [104]:
counter

767

In [106]:
sum_

2983

In [108]:
res

{'5406192366': 0,
 '7728013537': 5,
 '7710048970': 3,
 '7728168971': 7,
 '3801002781': 7,
 '7714877093': 2,
 '6829000028': 9,
 '4706006731': 0,
 '7725038220': 2,
 '7303024532': 0,
 '7701699414': 5,
 '5036011870': 7,
 '7736203789': 8,
 '7714572888': 2,
 '7705869699': 4,
 '7831001013': 6,
 '7750005450': 0,
 '7838395215': 0,
 '1435332211': 0,
 '2502039781': 0,
 '4715019631': 5,
 '7704612940': 3,
 '7809016423': 8,
 '6320007246': 4,
 '7704167721': 3,
 '7708063900': 0,
 '3812035570': 0,
 '7708813052': 1,
 '1435241444': 5,
 '7730231980': 3,
 '7202066550': 0,
 '2312054894': 4,
 '7725727149': 4,
 '6664013880': 8,
 '7750004231': 0,
 '7708709686': 5,
 '6454027396': 0,
 '7720413269': 2,
 '7705392230': 7,
 '7716051506': 4,
 '7727696432': 0,
 '7744000327': 5,
 '2204052762': 8,
 '5075011344': 2,
 '7731559044': 6,
 '5257116711': 0,
 '7703635416': 8,
 '7720261827': 3,
 '7702271396': 2,
 '3123011520': 4,
 '7708397772': 9,
 '7714056040': 4,
 '4345404971': 4,
 '7750004136': 0,
 '7750004030': 0,
 '78395023

Теперь можно посмотреть и попытаться смержить данные:

In [135]:
res 


{'7423004062': 15,
 '7703032986': 18,
 '6315232133': 22,
 '5405114781': 18,
 '8602190258': 13,
 '7744000951': 18,
 '8601999494': 16,
 '7831000122': 16,
 '2901009852': 14,
 '5502051657': 10,
 '7750004312': 11,
 '5902162552': 13,
 '6325064223': 10,
 '7536002161': 10,
 '7702073683': 23,
 '1700000350': 10,
 '7802084569': 11,
 '7728668727': 11,
 '7750005820': 12,
 '1326021671': 18,
 '7706628777': 12,
 '3525149310': 13,
 '5902198365': 11,
 '7709031643': 22,
 '7750004270': 10,
 '7725039953': 19,
 '1657023630': 21,
 '1433009894': 15,
 '7704300571': 15,
 '8602998609': 15,
 '7726445147': 18,
 '5902220892': 10,
 '7701100510': 19,
 '7815025049': 13,
 '7707050464': 24,
 '7707684460': 16,
 '2130058291': 12,
 '2460030985': 20,
 '2202000656': 12,
 '7449006730': 14,
 '7701041336': 14,
 '1326024785': 15,
 '7705880068': 13,
 '2801023444': 17,
 '1653003834': 14,
 '7728534829': 11,
 '8601023568': 22,
 '7725038124': 12,
 '7713056834': 22,
 '1650072068': 16,
 '2903000446': 19,
 '7744000038': 18,
 '1644001196